In [1]:
import subprocess
import os

import pandas as pd

import requests
from bs4 import BeautifulSoup

import json
pd.set_option('display.max_rows', 500)

![CRISP_DM](figures/CRISP_DM.png)

In [2]:
!cd

E:\Masters TUK\Sem 2\Enterprise data science\eps_covid_19\notebooks


# Data Understanding

We would like to track Corona VIRUS spread across countries and with personal local info


# GITHUB CSV DATA

git clone/pull:https://github.com/CSSEGISandData/COVID-19

In [3]:
git_pull=subprocess.Popen("/usr/bin/git pull",
                         cwd=os.path.dirname('../data/raw/COVID-19/'),
                         shell=True,
                         stdout=subprocess.PIPE,
                         stderr=subprocess.PIPE)
(out,error)= git_pull.communicate()

print("error:" + str(error))
print("out :"+str(out))

error:b'The system cannot find the path specified.\r\n'
out :b''


In [4]:
data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'

pd_raw=pd.read_csv(data_path)

In [5]:
pd_raw

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/17/22,5/18/22,5/19/22,5/20/22,5/21/22,5/22/22,5/23/22,5/24/22,5/25/22,5/26/22
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,179328,179477,179597,179624,179674,179716,179716,179771,179835,179835
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,275688,275732,275732,275732,275838,275864,275881,275939,275985,276012
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,265828,265834,265841,265847,265851,265854,265855,265860,265862,265864
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,42156,42572,42572,42572,42572,42572,42572,42572,42894,42894
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,99287,99287,99287,99287,99287,99287,99287,99433,99527,99527
5,NaN,Antarctica,-71.949900,23.347000,0,0,0,0,0,0,...,11,11,11,11,11,11,11,11,11,11
6,NaN,Antigua and Barbuda,17.060800,-61.796400,0,0,0,0,0,0,...,7795,7855,7910,7910,7942,7942,7982,7982,8062,8119
7,NaN,Argentina,-38.416100,-63.616700,0,0,0,0,0,0,...,9135308,9135308,9135308,9135308,9135308,9178795,9178795,9178795,9178795,9178795
8,NaN,Armenia,40.069100,45.038200,0,0,0,0,0,0,...,422917,422917,422917,422917,422917,422917,422939,422939,422939,422939
9,Australian Capital Territory,Australia,-35.473500,149.012400,0,0,0,0,0,0,...,121713,122718,123655,124477,125220,125828,126633,127556,128440,129263


In [6]:
pd_raw.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '5/17/22', '5/18/22', '5/19/22', '5/20/22', '5/21/22', '5/22/22',
       '5/23/22', '5/24/22', '5/25/22', '5/26/22'],
      dtype='object', length=860)

# Webscrapping

Data link :https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html

In [7]:
import requests
from bs4 import BeautifulSoup 

In [8]:
page=requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")

In [9]:
soup=BeautifulSoup(page.content,'html.parser')

In [10]:
html_table= soup.find('table')

In [11]:
all_rows=html_table.find_all('tr')

In [12]:
final_data_list=[]

In [13]:
for pos,rows in enumerate(all_rows):
    print(pos)
    #print(rows)
    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')]
    final_data_list.append(col_list)
    #for each_col in rows.find_all('td'):
        #print(each_col.get_text(strip=True))
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [14]:
pd_daily_status= pd.DataFrame(final_data_list).dropna().rename(columns={0:'State',
                                                                       1:'cases number',
                                                                       2:'difference to the previous day',
                                                                       3:'cases in the last 7 days',
                                                                       4:'7-day incidence',
                                                                       5:'deaths'})

In [15]:
pd_daily_status.head()

,State,cases number,difference to the previous day,cases in the last 7 days,7-day incidence,deaths
2,Baden-Württem­berg,4.219.824,15.664,79.502,"716,0",16.565
3,Bayern,5.625.401,21.500,110.380,"840,0",24.857
4,Berlin,1.178.540,3.057,15.099,"412,1",4.704
5,Branden­burg,877.357,2.460,12.866,"508,3",5.766
6,Bremen,238.474,830,4.751,"698,5",808


# REST API CALLS

In [16]:
data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronafälle_in_den_Bundesländern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [17]:
import json   #using json library because the data is in JSOn format

In [18]:
json_object=json.loads(data.content)

In [19]:
type(json_object)

dict

In [20]:
json_object.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'geometryType', 'spatialReference', 'fields', 'features'])

In [21]:
full_list=[]
for pos, each_dict in enumerate (json_object['features'][:]):
    full_list.append(each_dict['attributes'])

In [22]:
pd_full_list=pd.DataFrame(full_list)
pd_full_list.head()

,OBJECTID_1,LAN_ew_AGS,LAN_ew_GEN,LAN_ew_BEZ,LAN_ew_EWZ,OBJECTID,Fallzahl,Aktualisierung,AGS_TXT,GlobalID,faelle_100000_EW,Shape__Area,Shape__Length,Death,cases7_bl_per_100k,cases7_bl,death7_bl,cases7_bl_per_100k_txt,AdmUnitId
0,1,01,Schleswig-Holstein,Land,2910875,15,930445,1658959200000,01,fc5ba936-c95c-432c-8a33-9eb2f30b660f,31964.443681,4.573731e+10,2.881496e+06,2716,501.155151,14588,5,"501,2",1
1,2,02,Hamburg,Freie und Hansestadt,1852478,6,685167,1658959200000,02,0f3e860c-5181-4d3f-a421-1d51f50315ea,36986.512121,2.089396e+09,4.188002e+05,2815,359.248531,6655,6,"359,2",2
2,3,03,Niedersachsen,Land,8003421,9,2935136,1658959200000,03,3fd77024-c29b-4843-9be8-682ad48e60c9,36673.517487,1.299836e+11,4.008988e+06,9966,709.971398,56822,11,"710,0",3
3,4,04,Bremen,Freie Hansestadt,680130,5,238474,1658959200000,04,4132268b-54de-4327-ac1e-760e915112f1,35063.002661,1.119157e+09,3.357177e+05,808,698.542926,4751,0,"698,5",4
4,5,05,Nordrhein-Westfalen,Land,17925570,10,6227686,1658959200000,05,561d658f-3ee5-46e3-bc95-3528c6558ab9,34741.913367,8.782936e+10,2.648673e+06,26315,506.890436,90863,21,"506,9",5
